#### Procedure

Notes:
- 30k entries limit

1. use entries from 2010-2025
2. partition each naics code by month (180 iterations)
3. for each month download the corresponding csv file and store it into a 
4. after the loop concatenate all csv files into one file

### Imports

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import math
import time
import os
import polars as pl
import concurrent.futures

### Chrome Drive Setup

In [4]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
    return driver

### Shared Info

In [5]:
BASE_URL = 'https://www.fpds.gov/ezsearch/search.do?indexName=awardfull&templateName=1.5.3&s=FPDS.GOV&q='
year_one = 2000
month_intervals = [f'{math.floor(year_one+(i/12))}/{str((i%12)+1).zfill(2)}/01,{math.floor(year_one+((i+1)/12))}/{str(((i+1)%12)+1).zfill(2)}/01' 
                   for i in range(300)]
naics_codes = ['236220', '237310', '237990']
award_type = ['Definitive Contract']

# file directory locations

cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'Data')
documents_dir = os.path.dirname(cur_dir)
parent_dir = os.path.dirname(documents_dir)
downloads_dir = os.path.join(parent_dir, 'Downloads')

### Downloading Data Procedure

### FPDS
 
1. Iterate through every time interval...
 
- Input query to search
 
- Use CSV to download file
 
- Move + Rename File to Data folder
 
2. Concatenate all records to one data file

3. Remove all rows that satisfy notes above

4. For each unique contract...
    - Use contract ID in query
    - If multiple contracts select in page one with (0) appended (denotes original contract)
    - 

### USASpending (Run after FPDS)

1. Iterate through each processed dataframe (no mod. contracts)

### 236220 - Data FPDS (Not Completed)

In [33]:
# subsect by month
driver = driversetup()

for i in tqdm(range(145, len(month_intervals))):
    driver.get(BASE_URL)
    query = f'PRINCIPAL_NAICS_CODE:"{naics_codes[0]}" SIGNED_DATE:[{month_intervals[i]}] AWARD_TYPE:"{award_type[0]}"'
    # input query into text box
    driver.find_element("id", "searchText").send_keys(query)
    # click search button
    driver.find_element(By.XPATH, "/html/body/form/table/tbody/tr[2]/td[2]/input").click()
    # click on csv download button
    csv_box = driver.find_elements("id", "csvImg")
    if csv_box:
        csv_box[0].click()
        # 10 second time delay to wait for download 
        time.sleep(10)
        # rename the downloaded file
        old_file_name = os.path.join(downloads_dir, 'SearchResults.csv')
        new_file_name = f'{naics_codes[0]}_{month_intervals[i].replace("/","_")}.csv'
        # move the renamed downloaded file to the data folder buffer directory
        move_location = os.path.join(data_dir, new_file_name)
        os.rename(old_file_name, move_location)
    
driver.quit()


 85%|███████████████████████████████████████████████████████████████████▌            | 131/155 [26:01<04:57, 12.41s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 155/155 [30:33<00:00, 11.83s/it]


In [ ]:
# concatenate files

# 

### 236220 - Get Competition Info

In [6]:
BASE_URL = 'https://www.fpds.gov/ezsearch/search.do?indexName=awardfull&templateName=1.5.3&s=FPDS.GOV&q='

In [7]:
cur_dir = os.getcwd()
naics_data_dir = os.path.join(cur_dir, 'NAICS Processed')
df = pl.read_csv(os.path.join(naics_data_dir, '236220_second_proc.csv'))
df.head()

Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($)
str,i64,str,str,str,str,str,i64,i64,i64
"""GS11P14MAP0340""",236220,"""2014-10-16""","""4740""","""Z2AA""","""Southeast""","""RWDWFG6WGRK9""",644325,1,28426
"""IND0407CT66810""",236220,"""2006-12-21""","""1406""","""Y199""","""West""","""U4K9M66MUHR9""",68450,0,0
"""W912LD06C0036""",236220,"""2006-09-25""","""2100""","""Y111""","""Northeast""","""EATDZJL6JFJ5""",12573650,15,556384
"""GS11P06ZGC0339""",236220,"""2006-10-17""","""4740""","""J039""","""Southeast""","""QKCFMFL4MKT6""",11943,1,1504
"""HHSD200200618928C""",236220,"""2006-08-30""","""7523""","""Y111""","""Midwest""","""TTHNY8N2PQR3""",192500,1,38349


In [8]:
special_comp_params = ["Follow On to Competed Action",
                       "Not Available for Competition",
                       "Not Competed"]

In [1]:
# initial run through
driver = driversetup()
   
comp_type = []
bids = []

driver.get(BASE_URL)

for row in tqdm(df.iter_rows(named=True, start=0, end=2000), total=2000, desc="Processing rows"):
    contract_id = row['Contract ID']
    search_input = driver.find_element("id", "searchText")
    search_input.clear()  # clear input
    search_input.send_keys(contract_id)  # send new contract
    time.sleep(0.1)
    # click search button
    driver.find_element(By.XPATH, "/html/body/form/table/tbody/tr[2]/td[2]/input").click()
    original_window = driver.current_window_handle
    # get the original contract
    view_query = f'/html/body/table[4]/tbody/tr/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td/table[1]/tbody/tr[1]/td[2]/span/a[3]'
    try:
        driver.find_element(By.XPATH, view_query).click()
    except:
        comp_type.append('Error')
        bids.append(0)
        continue
    # switch tabs
    all_windows = driver.window_handles

    for window in all_windows:
        if window != original_window:
            driver.switch_to.window(window)
            break  # Switch to the newest tab and exit the loop

    # get extent competed
    try:
        extent = driver.find_element(By.CSS_SELECTOR, "#extentCompeted option:checked").text
    except:
        comp_type.append('Error')
        bids.append(0)
        continue
    
    # get number of bids
    bid_num = 1
    if extent not in special_comp_params: # if normal competition
        bid_num = driver.find_element(By.ID, "numberOfOffersReceived").get_attribute("value")

    # switch back to original window and close the others
    for window in driver.window_handles:
        if window != original_window:
            driver.switch_to.window(window)
            driver.close() 
            
    # add to lists
    comp_type.append('Open' if extent not in special_comp_params else 'Restricted')
    bids.append(bid_num)
            
    driver.switch_to.window(original_window)

NameError: name 'driversetup' is not defined

In [ ]:
# will need to make a new dataframe filtering out the errors

first_pass_df = df.with_columns([
    pl.col("Modification Count").fill_null(0),
    pl.col("Total Modified Action Obligation ($)").fill_null(0)
])